In [ ]:
import cv2
import cv2.aruco as aruco
from typing import Sequence, cast
import awkward as ak
from pathlib import Path
import numpy as np
from typing import Final
from matplotlib import pyplot as plt
from cv2.typing import MatLike

In [ ]:
A_PATH = Path("output/af_03.parquet")
B_PATH = Path("output/ae_08.parquet")

a_params = ak.from_parquet(A_PATH)[0]
b_params = ak.from_parquet(B_PATH)[0]
display(a_params)
display(b_params)

In [ ]:


def create_new_aruco_marker_origin(marker_length: float):
    """
    Create a new ArUco marker origin with the given length.

    0 -> x
    |
    v
    y

    0---1
    |   |
    3---2

    So that the center of the marker is the origin for this PnP problem.

    Args:
        marker_length: The length of the marker.
    """
    return np.array(
        [
            [-marker_length / 2, marker_length / 2, 0],
            [marker_length / 2, marker_length / 2, 0],
            [marker_length / 2, -marker_length / 2, 0],
            [-marker_length / 2, -marker_length / 2, 0],
        ]
    ).astype(np.float32)


DICTIONARY: Final[int] = aruco.DICT_4X4_50
# 400mm
MARKER_LENGTH: Final[float] = 0.4
aruco_dict = aruco.getPredefinedDictionary(DICTIONARY)
detector = aruco.ArucoDetector(
    dictionary=aruco_dict, detectorParams=aruco.DetectorParameters()
)

In [ ]:
a_img = cv2.imread(str(Path("dumped/marker/video-20241205-152716-board.png")))
a_mtx = ak.to_numpy(a_params["camera_matrix"])
a_dist = ak.to_numpy(a_params["distortion_coefficients"])

b_img = cv2.imread(str(Path("dumped/marker/video-20241205-152721-board.png")))
b_mtx =  ak.to_numpy(b_params["camera_matrix"])
b_dist = ak.to_numpy(b_params["distortion_coefficients"])

In [ ]:
a_corners, a_ids, _a_rejected = detector.detectMarkers(a_img)
b_corners, b_ids, _b_rejected = detector.detectMarkers(b_img)

In [ ]:
ok, a_rvec, a_tvec = cv2.solvePnP(create_new_aruco_marker_origin(MARKER_LENGTH), a_corners[0], a_mtx, a_dist)
if not ok:
    raise ValueError("Failed to solve PnP for A")
a_img_output =  cv2.drawFrameAxes(a_img, a_mtx, a_dist, a_rvec, a_tvec, MARKER_LENGTH)
plt.imshow(cv2.cvtColor(a_img_output, cv2.COLOR_BGR2RGB))

In [ ]:
ok, b_rvec, b_tvec = cv2.solvePnP(create_new_aruco_marker_origin(MARKER_LENGTH), b_corners[0], b_mtx, b_dist)
if not ok:
    raise ValueError("Failed to solve PnP for B")
b_img_output =  cv2.drawFrameAxes(b_img, b_mtx, b_dist, b_rvec, b_tvec, MARKER_LENGTH)
plt.imshow(cv2.cvtColor(b_img_output, cv2.COLOR_BGR2RGB))

In [ ]:
# Converts a rotation matrix to a rotation vector or vice versa
a_rmtx, _ = cv2.Rodrigues(a_rvec)
b_rmtx, _ = cv2.Rodrigues(b_rvec)
a_camera_coord = -(a_rmtx.T@ a_tvec)
b_camera_coord = -(b_rmtx.T @ b_tvec)
distance = np.linalg.norm(a_camera_coord - b_camera_coord)
a_distance = np.linalg.norm(a_camera_coord)
b_distance = np.linalg.norm(b_camera_coord)
display("d_ab={:.4}m a={:.4}m b={:.4}m".format(distance, a_distance, b_distance))
display("a_coord={}".format(a_camera_coord.T))
display("b_coord={}".format(b_camera_coord.T))

In [ ]:
def draw_grid(image: MatLike, rvec:MatLike, tvec:MatLike, camera_matrix:MatLike, dist_coeffs:MatLike, grid_size=10, grid_spacing=0.1):
    # Create grid points in marker coordinate system
    grid_points = []
    for i in range(-grid_size, grid_size+1):
        for j in range(-grid_size, grid_size+1):
            grid_points.append([i*grid_spacing, j*grid_spacing, 0])
    
    grid_points = np.array(grid_points, dtype=np.float32)

    # Project grid points onto image plane
    img_points, _ = cv2.projectPoints(grid_points, rvec, tvec, camera_matrix, dist_coeffs)
    img_points = img_points.reshape(-1, 2)

    # Draw grid lines
    for i in range(0, len(img_points), 2*grid_size+1):
        cv2.polylines(image, [img_points[i:i+2*grid_size+1].astype(int)], False, (0, 255, 0), 1)
    
    for i in range(2*grid_size+1):
        line_points = img_points[i::2*grid_size+1]
        cv2.polylines(image, [line_points.astype(int)], False, (0, 255, 0), 1)

    return image

In [ ]:
grid_image = draw_grid(a_img_output.copy(), a_rvec, a_tvec, a_mtx, a_dist, 10, 0.2)
plt.imshow(cv2.cvtColor(grid_image, cv2.COLOR_BGR2RGB))